# Generate MAE table

In [ ]:
import numpy as np
import pandas as pd
import os

## Config path

In [ ]:
BASE_PATH = '/root/work/datasets/artwork_sequence/'
RESULT_PATH = '/root/work/artwork_sequence/predicted_tours'
PLOT_PATH = '/root/work/artwork_sequence/plots'

In [ ]:
def get_predicted_tour_path(window_index):
    predicted_tour_base_path = os.path.join(RESULT_PATH,'config_'+str(window_index))

    predicted_tour_path = {
        'sequence_most_similar' : os.path.join(predicted_tour_base_path,'generated_sequence_based_previous_most_similar'),
        'sequence_rnn' : os.path.join(predicted_tour_base_path,'generated_sequence_rnn'),
       # 'sequence_rnn_multivariate' : os.path.join(predicted_tour_base_path,'generated_sequence_rnn_multivariate'),
        'sequence_rnn_code_embedding' : os.path.join(predicted_tour_base_path,'generated_sequence_rnn_code_embedding'),
        #'sequence_rnn_artist_code_embedding' : os.path.join(predicted_tour_base_path, 'generated_sequence_rnn_artist_code_embedding')


    }

    return predicted_tour_path

In [ ]:
window_indexes = list(range(3,9))

window_config = {}

for index in window_indexes:
    window_config[index] = get_predicted_tour_path(index)
    

## Load data

In [ ]:
def load_mae_data(predicted_tour_path):
    #load data
    mae_dict = {}
    for key, value in predicted_tour_path.items():
        mae_dict[key] = pd.read_csv(os.path.join(value,'mae.csv'))
    
    #Add model name to the dataframe
    for key, df in mae_dict.items():
        df['model'] = key

    #Concat all dataframe in one to plot MAE
    df_mae = pd.concat(list(mae_dict.values()))
    df_mae = df_mae.reset_index(drop=False)

    return df_mae

In [ ]:
window_mae = {}

for index, path in window_config.items():
    window_mae[index] = load_mae_data(path)

## Get mean and std by window

In [ ]:
window_mean_std = {}

for index, df in window_mae.items():
    #Compute mean and std
    df_mean_std = df.groupby('model')['mae'].agg(['mean', 'std'])
    df_mean_std = df_mean_std.reset_index()
    df_mean_std['window_size'] = index
    window_mean_std[index] = df_mean_std

In [ ]:
window_mean_std[3]

**Append all tables**

In [ ]:
df_all_mean_std = pd.concat(window_mean_std.values())
df_all_mean_std = df_all_mean_std.reset_index(drop=True)
df_all_mean_std.head()

**Save data**

In [ ]:
df_all_mean_std.to_excel(os.path.join(RESULT_PATH,'all_mean_std.xlsx'))